# 1. SymPy
[SymPy] is a exclusive and native library to Python, designed for [symbolic computation]. This mean mathematical expressions will be left in their symbolic forms and represented exactly, not approximately. SymPy can do this for all kinds of computations such as simplication, limits, derivatives and solving equations.

[SymPy]: https://github.com/sympy/sympy
[symbolic computation]: https://docs.sympy.org/latest/tutorials/intro-tutorial/intro.html#what-is-symbolic-computation

## 1.1. Expression
Expression is the most basic concept in Mathematics. We are going to use SymPy to create symbolic variables (distiguish them with Python variables), write expressions and substitute numbers to make ealuation.

In [52]:
import sympy as sym
from sspipe import p, px

In [50]:
# create some variables
x, y, z = sym.symbols('x,y,z')

In [21]:
# write an expression
expr = x**2 + 3*y + sym.cos(z)
expr

x**2 + 3*y + cos(z)

In [42]:
# substitute numbers, partially or fully
expr.subs({x: 2, y: 1})

cos(z) + 7

In [41]:
# by default, sympy will only evaluate non-decimal numbers once calling evalf()
sym.pi.evalf()

3.14159265358979

## 1.2. Simplification

In [16]:
expr = sym.sin(2*x)
sym.expand_trig(expr)

2*sin(x)*cos(x)

In [21]:
sym.factor(x**2 - 4*x + 3)

(x - 3)*(x - 1)

## 1.3. Solvers
First, go to a quick review of number sets in Mathematics: $\mathbb{N}$ (naturals), $\mathbb{Z}$ (integers), $\mathbb{Q}$ (rationals), $\mathbb{R}$ (reals) and $\mathbb{C}$ (complexes).

In [6]:
import sympy as sym
from sspipe import p, px

In [54]:
x, y, z = sym.symbols('x:z')

### Equation solver
The function
<code style='font-size:13px'><a href='https://docs.sympy.org/latest/modules/solvers/solveset.html'>solveset()</a></code>
solves an equation for one variable and returns a set of solutions. When an expression is passed in, it is automatically assumed to equal $0$. We can also change the equation domain to get appropriate solutions.

In [71]:
sym.Eq(x**2, 1) | p(sym.solveset, x)

{-1, 1}

In [85]:
'x^2 + 2*x + 5' | p(sym.solveset, x)

{-1 - 2*I, -1 + 2*I}

In [86]:
'x^2 + 2*x + 5' | p(sym.solveset, x, domain=sym.S.Reals)

EmptySet

### Linear system solver
We can solve systems of equations by specifying each equation explicitly or using agumented matrix.

In [88]:
eqSys = [
    sym.Eq(x + y + z, 6),
    sym.Eq(x + 2*y + z, 8),
    sym.Eq(x + y + 2*z, 9)
]

sym.linsolve(eqSys, [x, y, z])

{(1, 2, 3)}

In [89]:
matAgumented = sym.Matrix([
    [1, 1, 1, 6],
    [1, 2, 1, 8],
    [1, 1, 2, 9]
])

sym.linsolve(eqSys, [x, y, z])

{(1, 2, 3)}

### Inequality solver
SymPy [solves inequalities] via a number of functions adapt to various cases.

[solves inequalities]: https://docs.sympy.org/latest/modules/solvers/inequalities.html

# 2. Limits

## 2.1. Preview

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(['seaborn', 'seaborn-whitegrid'])
%config InlineBackend.figure_format = 'retina'

### The velocity problem
[Rate of change] or instantaneous [velocity] is an important concept in Data Science and Machine Learning. It is expressed as the [slope] $m$ of a line $y=mx+c$. However, in a non-linear function (which is very popular), the velocity changes as $x$ changes. To measure the instantaneous velocity at any given point, an visual approach is trying to draw a [tangent] then find its slope by computing $\Delta{y}/\Delta{x}$.

<img src='image/velocity_problem.png' style='height:200px; margin:20px auto 20px;'>

Let's an this example, we want to draw the tangent of the parabola at the blue point, $a$. It is the blue line, but we will have a hard time finding it. It would be more reasonale for us to approximate the tangent by drawing a [secant], like the red one.

$$m_a\approx\frac{f(x)-f(a)}{x-a}$$

We also observe that the orange secant is less accurate, as the orange $x$ is further from the $a$, compared to red $x$. In other words, we would want to move $x$ as close to $a$ as possible, but never let them overlap to make sure the secant can always be drawn. We say that $x$ approaches $a$, denoted $x\rightarrow a$. Such a process is mathematically written as:

$$m_a=\lim_{x\rightarrow a}\frac{f(x)-f(a)}{x-a}$$

[Rate of change]: https://en.wikipedia.org/wiki/Time_derivative
[velocity]: https://en.wikipedia.org/wiki/Velocity
[slope]: https://en.wikipedia.org/wiki/Slope
[tangent]: https://en.wikipedia.org/wiki/Tangent
[secant]: https://en.wikipedia.org/wiki/Secant_line

### The area problem
Another popular problem in Calculus is evaluating the area under a curve over an interval. Same as the velocity problem, we cannot compute it exactly, but can try to approximate it.

<img src='image/area_problem.png' style='height:350px; margin:20px auto 20px;'>

One way to do this is dividing up the interval $[a,b]$ into $N$ small bins, each has a width of $h$ and is represented by a point $x_n$. This point is usually placed at the center or at one of the two boundaries of a bin. For each bin $n$, draw a rectangular having width of $h$ and height of $f(x_n)$. The area can be approximated by summing up the areas of all rectangulars:

$$S\approx\sum_{n=1}^{N}{hf(x_n)}$$

It's obvious that, the smaller $h$ is, the more accurate of our approximation. So we would want to reduce $h$ to be as close to $0$ as possible, but never equal to $0$ because at this point we cannot *draw* rectangulars. This is formally written as:

$$S=\lim_{h\rightarrow0}\sum_{n=1}^{N}{hf(x_n)}$$

## 2.2. Limits

### One-sided limit
Now let's extend the understanding of limits using three examples below.

<img src='https://openstax.org/apps/archive/20220815.182343/resources/b2affeb36b502a8fb8fd7d3f6f7314d1dcb25614' style='height:300px; margin:20px auto 20px;'>

In [1]:
import sympy as sym
from sspipe import p, px

In [10]:
x = sym.Symbol('x')
sym.sin(1/x) | p(sym.limit, x, 0, dir='+')

AccumBounds(-1, 1)

# 3. Derivatives
The [derivative] of a function $f$ is a function $f'$ describes velocities at different points. The process of finding a derivative is called [differentiation]. In Data Science, the most recognizable application of Differential Calculus is solving optimization problems.

[derivative]: https://en.wikipedia.org/wiki/Derivative
[differentiation]: https://en.wikipedia.org/wiki/Differential_calculus

# 4. Integrals
The [integral] of a function $f$ is a function $F$ describes areas under the curve up to different points. The process of finding an integral is call [integration]. In Data Science, the most recognizable application of Integral Calculus is defining probabilities.

[integral]: https://en.wikipedia.org/wiki/Antiderivative
[integration]: https://en.wikipedia.org/wiki/Integral

# References
- *openstax.org - [Calculus Volume 1](https://openstax.org/books/calculus-volume-1/pages/1-introduction)*
- *openstax.org - [Calculus Volume 2](https://openstax.org/books/calculus-volume-2/pages/1-introduction)*
- *openstax.org - [Calculus Volume 3](https://openstax.org/books/calculus-volume-3/pages/1-introduction)*
- *rodrigopacios.github.io - [Thomas' Calculus](https://rodrigopacios.github.io/mrpacios/download/Thomas_Calculus.pdf)*